## Visualization of Inference-based Analyses 
We visualize results of inference-based analyses. By Inference-based analyses we mean analyses which extract characteristics of the data by applying language models to them or in general inference-based processes. 

### 1. Query Intent
We apply an intent classifier to the queries. The model classifies the data into the categories *informational*, *navigational* and *transactual*. 


In [ ]:
from thesis_schneg.model import DatasetName

##### set visualization configuration #####

# combine all plots in one figure or plot each analysis in a single figure
single_plot = True
# single_plot = False

# save visualization or not
# save_vis: bool = False
save_vis: bool = True

# select dataset (if none is selected all datasets are visualized in a joint plot) 
dataset: DatasetName = None 
# dataset: DatasetName = 'aol'
# dataset: DatasetName = 'aql'
# dataset: DatasetName = 'ms-marco'
# dataset: DatasetName = 'orcas'

analyses = []
# set analysis that should be visualized
analyses.append('query-intent')

# test_data = True
test_data = False

normalize_data = True
# normalize_data = False

# take_new_directory = True
take_new_directory = False



In [ ]:
from thesis_schneg.vis_modules import _get_results_paths, load_results
import time
color = None
label = None
# load data
analysis_data = []
for analysis_name in analyses:
    result_data = {}
    # crawl files from all datasets and load into dictionary

    paths = {f"{name}": _get_results_paths(name, analysis_name, take_new_directory) for name in [
            "aol", "aql", "ms-marco", "orcas"]}
    # iterate over datasets and create visualization
    for name, result_paths in paths.items():
        start_time = time.time()
        print(f"Loading data from {name} dataset")
        vis_data = load_results(result_paths, test_data=test_data)
        # change the expression 'Abstain' to 'Informational' of the 'query-intent' column
        vis_data['query-intent'] = vis_data['query-intent'].replace('Abstain', 'Informational')
        result_data.update({name: vis_data})
        end_time = time.time()  
        print(f"{name} loaded in {(end_time - start_time)/60} min")
    label=["AOL", "AQL", "MS-MARCO", "ORCAS"]
    analysis_data.append(result_data)


In [ ]:
for item in analysis_data:
    for key, value in item.items():
        print(key)
        print(value)
        print(f"{value['count()'].sum()/1e6} x 10^6 queries")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import scienceplots
from pathlib import Path

# save_vis = False
save_vis = True
# plot_format = 'pgf'
plot_format = 'pdf'


# set size of plot
textwidth = 5.5129
aspect_ratio = 6/8
scale = 1.0
width = textwidth * scale
height = width * aspect_ratio * 0.8

# use science style for plots from scienceplots library
plt.style.use(["science", "ieee"]) #"grid", "ieee",


color = ['tab:blue', 'tab:orange', 'tab:gray', 'tab:red']


fig,axes = plt.subplots(ncols=2, nrows=2, figsize=(width, height))

# iterate over axes and plot data. Only include the names of the bars for the plots in the first column
for i, ax in enumerate(axes.flatten()):
    # plot data
    data = result_data[list(result_data.keys())[i]]
    data = data.sort_values(by='query-intent', ascending=True)
    data['count()'] = data['count()'] / data['count()'].sum()
    # hrizontal bar plot
    ax.bar(data['query-intent'], data['count()'], color=color[i], alpha=0.5)
    # Add legend with dataset name
    dataset_name = list(result_data.keys())[i].upper()
    if dataset_name == 'MS-MARCO':
        dataset_name = 'MS-MARCO WS'
    ax.legend([dataset_name], loc='upper right', fontsize=8)
    # Only show y-axis labels for the first column
    if i < 2:  # If the plot is in the second column
        ax.set_xticklabels([])  # Remove y-axis labels


fig.supxlabel('Query Intent', y=0.99)
fig.supylabel('Frequency', x=0.03)
plt.tight_layout()
# either save visualization or show it
plt.show()

if save_vis:
    vis_dir = Path(
            f"/home/benjamin/studium/master/masterarbeit/thesis-schneg/plots/{analysis_name}")
    # make directory if it does not exist
    if not vis_dir.exists():
        vis_dir.mkdir(parents=True)
    # delete old plot
    for file in vis_dir.iterdir():
        if file.is_file() and file.name.endswith(f'.{plot_format}'):
            file.unlink()

    fig.savefig(vis_dir.joinpath(f"all.{plot_format}"), format=plot_format)



### 2. PII Entities

We have extracted the labels from all PII Entities of the query logs and present their distribution here:

In [ ]:
from thesis_schneg.model import DatasetName
from thesis_schneg.vis_modules import _get_results_paths, load_results
import time

analysis_name = 'group-presidio-pii'

dataset = None
# dataset = 'aol'
# dataset = 'aql'
# dataset = 'ms-marco'
# dataset = 'orcas'


if dataset is None:
    result_data = {}

    # crawl files from all datasets and load into dictionary
    paths = {f"{name}": _get_results_paths(name, analysis_name, cleaned_aql=True) for name in [
        "aol", "aql", "ms-marco", "orcas"]}
    # iterate over datasets and create visualization
    for name, result_paths in paths.items():
        start_time = time.time()
        print(f"Loading data from {name} dataset")
        vis_data = load_results(result_paths)
        result_data.update({name: vis_data})
        end_time = time.time()  
        print(f"{name} loaded in {(end_time - start_time)/60} min")
else:
    result_data = {}
    path = _get_results_paths(dataset, analysis_name, cleaned_aql=True)
    vis_data = load_results(path)
    result_data.update({dataset: vis_data})

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import scienceplots
from pathlib import Path

dataset_sizes = {
    'aol': 36389567,
    'aql': 273985982, # cleaned aql
    'ms-marco': 9224790,
    'orcas': 10405311
}      

save_vis = False
# save_vis = True
# plot_format = 'pgf'
plot_format = 'pdf'

# set size of plot
textwidth = 5.5129
aspect_ratio = 6/8
scale = 1.0
width = textwidth * scale
height = width * aspect_ratio * 0.45

# use science style for plots from scienceplots library
plt.style.use(["science", "ieee"]) #"grid", "ieee",


color = ['tab:blue', 'tab:orange', 'tab:gray', 'tab:red']
datasets = ['aol', 'aql', 'ms-marco web search', 'orcas']

fig,ax = plt.subplots(ncols=1, nrows=1, figsize=(width, height))

# iterate over axes and plot data. Only include the names of the bars for the plots in the first column
for i in range(len(result_data)):


    freq = result_data[list(result_data.keys())[i]]['count()'].sum() / dataset_sizes[list(result_data.keys())[i]] 
    ax.barh( i, freq, color=color[i], alpha=0.5)
    # ax.bar( i, freq, color=color[i], alpha=0.5)


# Add a single legend for all datasets
dataset_names = [name.upper() for name in result_data.keys()]
dataset_names = [name.replace('MS-MARCO', 'MS-MARCO WS') for name in dataset_names]

legend = fig.legend(
    dataset_names,
    loc='upper left',
    # bbox_to_anchor=(0.795, 0.95),
    bbox_to_anchor=(-0.15, 0.8),

    fontsize=8,
    # frameon=True,  # Enable the frame
    edgecolor='black',  # Set the frame color
    framealpha=1,  # Set the transparency of the frame (1 = opaque)
    fancybox=False,  # Disable rounded corners (optional)
)
ax.set_yticklabels([])  # Remove x-axis labels

legend.get_frame().set_linewidth(0.5)  # Set the frame width


fig.supxlabel('Frequency', y=0.99)
fig.supylabel('PII-Entity-Ratio', x=1, y=0.6)
plt.tight_layout()
# either save visualization or show it
plt.show()

if save_vis:
    vis_dir = Path(
            "/home/benjamin/studium/master/masterarbeit/thesis-schneg/plots/pii-ratio")
    # make directory if it does not exist
    if not vis_dir.exists():
        vis_dir.mkdir(parents=True)
    # delete old plot
    for file in vis_dir.iterdir():
        if file.is_file() and file.name.endswith(f'.{plot_format}'):
            file.unlink()

    fig.savefig(vis_dir.joinpath(f"all.{plot_format}"), format=plot_format)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import scienceplots
from pathlib import Path

# save_vis = False
save_vis = True
# plot_format = 'pgf'
plot_format = 'pdf'


# set size of plot
textwidth = 5.5129
aspect_ratio = 6/8
scale = 1.0
width = textwidth * scale
height = width * aspect_ratio

# use science style for plots from scienceplots library
plt.style.use(["science", "ieee"]) #"grid", "ieee",


color = ['tab:blue', 'tab:orange', 'tab:gray', 'tab:red']

# Extract all labels from the result data and find the intersection of all labels in all datasets
all_labels = None
for name, vis_data in result_data.items():
    labels = set(vis_data['entity-label'].unique())
    if all_labels is None:
        all_labels = labels
    else:
        all_labels = all_labels.intersection(labels)

# convert labels back into one list
all_labels = list(all_labels)


fig,axes = plt.subplots(ncols=2, nrows=2, figsize=(width, height))

# iterate over axes and plot data. Only include the names of the bars for the plots in the first column
for i, ax in enumerate(axes.flatten()):
    # plot data
    data = result_data[list(result_data.keys())[i]]
    data = data[data['entity-label'].isin(all_labels)]
    data = data.sort_values(by='entity-label', ascending=True)
    data['count()'] = data['count()'] / data['count()'].sum()
    # hrizontal bar plot
    ax.barh(data['entity-label'], data['count()'], color=color[i], alpha=0.5)
    # Add legend with dataset name
    dataset_name = list(result_data.keys())[i].upper()
    if dataset_name == 'MS-MARCO':
        dataset_name = 'MS-MARCO WS'
    ax.legend([dataset_name], loc='upper right', fontsize=8)

    # Only show y-axis labels for the first column
    if i % 2 != 0:  # If the plot is in the second column
        ax.set_yticklabels([])  # Remove y-axis labels



fig.supxlabel('Frequency', y=0.99)
fig.supylabel('PII Entity Label', x=0.03)
plt.tight_layout()
# either save visualization or show it
plt.show()

if save_vis:
    vis_dir = Path(
            f"/home/benjamin/studium/master/masterarbeit/thesis-schneg/plots/{analysis_name}")
    # make directory if it does not exist
    if not vis_dir.exists():
        vis_dir.mkdir(parents=True)
    # delete old plot
    for file in vis_dir.iterdir():
        if file.is_file() and file.name.endswith(f'.{plot_format}'):
            file.unlink()

    fig.savefig(vis_dir.joinpath(f"all.{plot_format}"), format=plot_format)

## 3. Question Classifier
We employ a clssifier to detect questions in the data. The classifier is intended for english text, hence we apply it only on the english subsets of MS MARCO and AQL.

In [ ]:
from thesis_schneg.model import DatasetName
from thesis_schneg.vis_modules import _get_results_paths, load_results
import time

analysis_name = 'questions'

dataset = None
# dataset = 'aol'
# dataset = 'aql'
# dataset = 'ms-marco'
# dataset = 'orcas'


if dataset is None:
    result_data = {}

    # crawl files from all datasets and load into dictionary
    paths = {f"{name}": _get_results_paths(name, analysis_name, english=True) for name in [
        "aol", "aql", "ms-marco", "orcas"]}
    # iterate over datasets and create visualization
    for name, result_paths in paths.items():
        start_time = time.time()
        print(f"Loading data from {name} dataset")
        vis_data = load_results(result_paths)
        result_data.update({name: vis_data})
        end_time = time.time()  
        print(f"{name} loaded in {(end_time - start_time)/60} min")
else:
    result_data = {}
    path = _get_results_paths(dataset, analysis_name, english=True)
    vis_data = load_results(path)
    result_data.update({dataset: vis_data})

print(result_data)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import scienceplots
from pathlib import Path

# save_vis = False
save_vis = True
# plot_format = 'pgf'
plot_format = 'pdf'


# set size of plot
textwidth = 5.5129
aspect_ratio = 6/8
scale = 1.0
width = textwidth * scale *0.7
height = width * aspect_ratio * 0.6

# use science style for plots from scienceplots library
plt.style.use(["science", "ieee"]) #"grid", "ieee",


color = ['tab:blue', 'tab:orange', 'tab:gray', 'tab:red']
datasets = ['aol', 'aql', 'ms-marco web search', 'orcas']
# Extract all labels from the result data and find the intersection of all labels in all datasets


fig,ax = plt.subplots(ncols=1, nrows=1, figsize=(width, height))

# iterate over axes and plot data. Only include the names of the bars for the plots in the first column
for i in range(len(result_data)):

    # plot data
    data = result_data[list(result_data.keys())[i]]
    data['count()'] = data['count()'] / data['count()'].sum()
    data = data.query('`is-question` == True')
    # hrizontal bar plot
    # ax.bar(i, data['count()'], color=color[i], alpha=0.5)
    ax.barh(i, data['count()'], color=color[i], alpha=0.5)

    # Add legend with dataset name
    # ax.legend([datasets[i].upper()], loc='upper left', fontsize=8)

# Add a single legend for all datasets
dataset_names = [name.upper() for name in result_data.keys()]
dataset_names = [name.replace('MS-MARCO', 'MS-MARCO WS') for name in dataset_names]

legend = fig.legend(
    dataset_names,
    loc='upper left',
    # bbox_to_anchor=(0.795, 0.95),
    bbox_to_anchor=(-0.23, 0.8),

    fontsize=8,
    # frameon=True,  # Enable the frame
    edgecolor='black',  # Set the frame color
    framealpha=1,  # Set the transparency of the frame (1 = opaque)
    fancybox=False,  # Disable rounded corners (optional)
)
ax.set_yticklabels([])  # Remove x-axis labels

legend.get_frame().set_linewidth(0.5)  # Set the frame width


fig.supxlabel('Frequency', y=0.99)
fig.supylabel('Question-Ratio', x=1, y=0.6)

# legend = fig.legend(
#     dataset_names,
#     loc='upper left',
#     bbox_to_anchor=(0.158, 0.92),
#     fontsize=8,
#     frameon=True,  # Enable the frame
#     edgecolor='black',  # Set the frame color
#     framealpha=1,  # Set the transparency of the frame (1 = opaque)
#     fancybox=False,  # Disable rounded corners (optional)
# )
# ax.set_xticklabels([])  # Remove x-axis labels

# legend.get_frame().set_linewidth(0.5)  # Set the frame width


# fig.supxlabel('Question-Ratio', y=0.99)
# fig.supylabel('Frequency', x=0.03)
plt.tight_layout()
# either save visualization or show it
plt.show()

if save_vis:
    vis_dir = Path(
            f"/home/benjamin/studium/master/masterarbeit/thesis-schneg/plots/{analysis_name}")
    # make directory if it does not exist
    if not vis_dir.exists():
        vis_dir.mkdir(parents=True)
    # delete old plot
    for file in vis_dir.iterdir():
        if file.is_file() and file.name.endswith(f'.{plot_format}'):
            file.unlink()

    fig.savefig(vis_dir.joinpath(f"all.{plot_format}"), format=plot_format)